In [10]:
from heapq import heappush, heappop
import itertools

MAX_SALES = 8
STOCK_PRICES = [51, 10, 1, 10, 1, 10, 6, 76, 67, 78, 65, 43, 23, 22, 3, 43, 65, 56,
                57, 32, 3, 65, 65, 22, 2, 29]

partialSolutions = []

class PartialSolution():
    def __init__(self, known_sales, next_sale=None):
        self.known_sales = known_sales[::]
        if next_sale:
            self.known_sales.append(next_sale)
        self.value = self._calc_value()
        self.upper_limit = self._calc_upper_limit()
        
    def enumerate_children(self):
        used_dates = set(itertools.chain.from_iterable(self.known_sales))
        
        free_dates = set(range(len(STOCK_PRICES))) - used_dates
        free_dates = sorted(list(free_dates))
        
        
        for buy_date in free_dates[:-1]:
            for sell_date in free_dates:
                if sell_date > buy_date:
                    yield (buy_date, sell_date)
    
    def _calc_value(self):
        value = 0
        for buy_date, sell_date in self.known_sales:
            value += STOCK_PRICES[sell_date] - STOCK_PRICES[buy_date]
        return value
    
    def _calc_upper_limit(self):
        """
        This upper limit works if we make the following assumptions:
         - When num_sales is zero, we are the only element in the queue
         - The same solution can be achieved through multiple paths (that is,
           the same set of sales can be achieved in different order).  We only
           explore the branches that have the highest value sales first
        """
        try:
            return self.value * float(MAX_SALES ) / float(self.num_sales())
        except ZeroDivisionError:
            return 0
    
    def num_sales(self):
        return len(self.known_sales)
    
    def __repr__(self):
        return "{} -> {} (bound: {})".format(str(self.value), str(self.known_sales), int(self.upper_limit))
    
    def __lt__(self, other):
        return self.value > other.value

value_maximum = 0
value_maximizer = PartialSolution([])

partialSolutions.append(value_maximizer)

from pprint import pprint

while partialSolutions:
    #pprint(partialSolutions)
    sol = heappop(partialSolutions)
    if sol.upper_limit < value_maximum:
        break
    if sol.num_sales() >= MAX_SALES:
        continue
    
    for next_sale in sol.enumerate_children():
        next_child = PartialSolution(sol.known_sales, next_sale)
        if next_child.value > value_maximum:
            print("New max: ", next_child)
            value_maximum = next_child.value
            value_maximizer = next_child
        heappush(partialSolutions, next_child)

New max:  25 -> [(0, 7)] (bound: 200)
New max:  27 -> [(0, 9)] (bound: 216)
New max:  66 -> [(1, 7)] (bound: 528)
New max:  68 -> [(1, 9)] (bound: 544)
New max:  75 -> [(2, 7)] (bound: 600)
New max:  77 -> [(2, 9)] (bound: 616)
New max:  102 -> [(2, 9), (0, 7)] (bound: 408)
New max:  143 -> [(2, 9), (1, 7)] (bound: 572)
New max:  152 -> [(2, 9), (4, 7)] (bound: 608)
New max:  168 -> [(2, 9), (4, 7), (0, 8)] (bound: 448)
New max:  209 -> [(2, 9), (4, 7), (1, 8)] (bound: 557)
New max:  213 -> [(2, 9), (4, 7), (6, 8)] (bound: 568)
New max:  214 -> [(2, 9), (4, 7), (14, 16)] (bound: 570)
New max:  230 -> [(2, 9), (4, 7), (14, 16), (0, 8)] (bound: 460)
New max:  271 -> [(2, 9), (4, 7), (14, 16), (1, 8)] (bound: 542)
New max:  275 -> [(2, 9), (4, 7), (14, 16), (6, 8)] (bound: 550)
New max:  276 -> [(2, 9), (4, 7), (14, 16), (20, 21)] (bound: 552)
New max:  292 -> [(2, 9), (4, 7), (14, 16), (20, 21), (0, 8)] (bound: 467)
New max:  333 -> [(2, 9), (4, 7), (14, 16), (20, 21), (1, 8)] (bound: 53